In [ ]:
from nb_shared import *

### Cluster for batch processing

In [ ]:
cluster = start_cluster(n_workers=20, local=False)

## Target Zones

In [ ]:
target_ch5ids = [
    # 'ANBC2',
    # 'RCYC2',
    'ALEC2',
    # 'ENMC2',
    # 'WBRW4',
    'BPPA3',
    # 'OAWU1'
]

## Snow-17 SWE

In [ ]:
snow17_db = SweDB(SNOW_17_DB_CONNECTION)

## SWANN for CBRFC zone

In [ ]:
zone_db = ZoneDB(DB_CONNECTION_INFO)
swann_xr = xr.open_mfdataset(SWANN_HOME_DIR + '*.nc', parallel=True)

# Get SWE for each year

peak_swe = peak_swe_for_swann(swann_xr, target_zone_names, zone_db) 

In [ ]:
target_zones = zone_db.zone_in_ch5_ids(target_ch5ids)

In [ ]:
swann_swe = {
    cz.zone: swann_swe_for_zone(swann_xr, cz.zone, zone_db) for cz in target_zones
}

In [ ]:
cluster.shutdown()

In [ ]:
snow17_swe = {
    cz.zone: Swe(snow17_db, cz.ch5_id + 'H', cz.zone).dataframe for cz in target_zones
}

In [ ]:
def plot_zone(ch5_id):
    fig, (ax1, ax2) = plt.subplots(2, 1, dpi=300, sharex=True, figsize=(12,5))
    fig.subplots_adjust(hspace=0)
    colors = ['cadetblue', 'goldenrod', 'crimson'] 

    for index, zone in enumerate(['HLF', 'HMF', 'HUF']):
        s17 = snow17_swe[ch5_id + zone].loc['1999-10-01':'2020-07-31']
        swann = swann_swe[ch5_id + zone].loc['1999-10-01':'2020-07-31']
        diff = s17['SWE (mm)'] - swann[ch5_id + zone + '_SWANN']
        
        ax1.plot(
            s17.index, s17['SWE (mm)'], label=f'{zone}', color=colors[index], alpha=0.6,
        )
        # ax.plot(
        #     swann.index, swann[zone_name + '_SWANN'], 
        #     label='UA SWE', ls=(0, (5, 1)), lw=0.8, color='goldenrod'
        # )
        ax2.plot(
            s17.index, diff, label=f'{zone}', lw=0.9, color=colors[index], alpha=0.7,
        )
        if zone == 'HUF':
            ax1.set_ylim(bottom=-100, top=math.ceil(s17['SWE (mm)'].max() + 20))
            ax2.set_xlim(left=s17.index[0], right=s17.index[-1])

    ax2.axhline(0, color='black', ls='dashed', lw=0.6)
    bbox = dict(boxstyle="round4,pad=0.3", fc="powderblue", lw=0)
    ax2.annotate('SNOW 17 higher', xy=(0.01, 0.9), fontstyle='italic', xycoords='axes fraction', bbox=bbox)
    bbox = dict(boxstyle="round4,pad=0.3", fc="thistle", lw=0)
    ax2.annotate('UA SWE higher', xy=(0.01, 0.05), fontstyle='italic', xycoords='axes fraction', bbox=bbox)

    ax1.set_ylabel(r"SNOW-17 SWE (mm)")
    ax1.legend(loc='upper left', ncols=3, frameon=False)
    
    ax2.set_ylabel(r"$\Delta$ SWE (mm)")
    ax2.set_xlabel('Year') 

In [ ]:
plot_zone('ALEC2')

In [ ]:
import math

In [ ]:
def plot_zone(ch5_id):
    fig, (ax1, ax2) = plt.subplots(2, 1, dpi=300, sharex=True, figsize=(12,5))
    fig.subplots_adjust(hspace=0)
    colors = ['cadetblue', 'goldenrod', 'crimson'] 

    for index, zone in enumerate(['HLH', 'HMH', 'HUH']):
        s17 = snow17_swe[ch5_id + zone].loc['1999-10-01':'2020-07-31']
        swann = swann_swe[ch5_id + zone].loc['1999-10-01':'2020-07-31']
        diff = s17['SWE (mm)'] - swann[ch5_id + zone + '_SWANN']
        
        ax1.plot(
            s17.index, s17['SWE (mm)'], label=f'{zone}', color=colors[index], alpha=0.6,
        )
        # ax.plot(
        #     swann.index, swann[zone_name + '_SWANN'], 
        #     label='UA SWE', ls=(0, (5, 1)), lw=0.8, color='goldenrod'
        # )
        ax2.plot(
            s17.index, diff, label=f'{zone}', lw=0.9, color=colors[index], alpha=0.7,
        )
        if zone == 'HUH':
            ax1.set_ylim(bottom=-25, top=math.ceil(s17['SWE (mm)'].max() + 20))
            ax2.set_xlim(left=s17.index[0], right=s17.index[-1])

    ax2.axhline(0, color='black', ls='dashed', lw=0.6)
    bbox = dict(boxstyle="round4,pad=0.3", fc="powderblue", lw=0)
    ax2.annotate('SNOW 17 higher', xy=(0.01, 0.9), fontstyle='italic', xycoords='axes fraction', bbox=bbox)
    bbox = dict(boxstyle="round4,pad=0.3", fc="thistle", lw=0)
    ax2.annotate('UA SWE higher', xy=(0.01, 0.05), fontstyle='italic', xycoords='axes fraction', bbox=bbox)
    
    ax1.set_ylabel(r"SNOW-17 SWE (mm)")
    # ax1.set_title(ch5_id)
    ax1.legend(loc='upper left', ncols=3, frameon=False)
    
    ax2.set_ylabel(r"$\Delta$ SWE (mm)")
    ax2.set_xlabel('Year') 

In [ ]:
plot_zone("BPPA3")

ZonePlotter.show_all(plot, 1)

ZonePlotter.save_html(HTML_OUTPUT + 'virgin_peak_SWE', plot, 1)